In [1]:
import loop_utils as u
import make_atomic_files as f
import pandas as pd
import metrics
import os
from recbole.config import Config
from recbole.utils import init_seed
from recbole.data import data_preparation, create_dataset
from recbole.quick_start.quick_start import load_data_and_model, get_model, get_trainer

In [2]:
params = {'dynamic': False,
 'embedding_size': 64,
 'learning_rate': 0.0008458746521763345,
 'mlp_hidden_size': '[64,64,64]',
 'sample_num': 1,
 'train_batch_size': 4096,
 'dataset': 'foursquare_train'}

In [4]:
model = u.training('BPR', params)

In [3]:
config_test = {
        'dataset': 'foursquare', 
        'data_path': os.getcwd(), 
        'train_neg_sample_args': None
}

config = Config(model='BPR', config_file_list=['dataset_config.yaml'], config_dict = config_test)

# init random seed
init_seed(config['seed'], config['reproducibility'])

# dataset creating and filtering
dataset = create_dataset(config)
#fixing train and validation set
train_set,validation_set,test_set = data_preparation(config, dataset)

In [6]:
tt, vv, ss = train_set._dataset.leave_one_out(group_by='uid', leave_one_mode = 'valid_and_test')
from recbole.trainer import Trainer

In [7]:
tt

foursquare
The number of users: 1084
Average actions of users: 78.0
The number of items: 27517
Average actions of items: 3.451017239970586
The number of inters: 84474
The sparsity of the dataset: 99.71680036239255%
Remain Fields: ['uid', 'venue_id', 'timestamp', 'venue_category_name', 'neg_venue_id', 'neg_venue_category_name']

In [25]:
model = get_model(config['model'])(config, tt).to(config['device'])
# trainer loading and initialization
trainer = Trainer(config, model)

# model training
best_valid_score, best_valid_result = trainer.fit(tt, vv)

KeyError: 'neg_venue_id'

In [8]:
model = u.training('BPR', params, train_set, validation_set)

In [9]:
model.evaluate(test_set)

OrderedDict([('hit@10', 0.8827), ('precision@10', 0.2291)])